<a href="https://colab.research.google.com/github/abbycakes02/Perceptron/blob/main/neuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post7-py3-none-any.whl size=2950 sha256=7bbfe86526383b3e0e11a4c55f3cd7db2fd355aefabe0873345d6658a0616ab2
  Stored in directory: /root/.cache/pip/wheels/c8/9c/85/72901eb50bc4bc6e3b2629378d172384ea3dfd19759c77fd2c
Successfully built sklearn


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [ ]:
#the key thing is we are passing the # of hidden layer nodes as a parameter. We are going to use this function in order to experiment
#with how altering the # of hidden layer nodes impacts our performance.

def create_model(hidden_layer_nodes, input_dimensions, output_dimensions):
  #make a sequential network that feeds information forward and back propogates
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_layer_nodes, activation = 'relu', input_dim = input_dimensions),
      tf.keras.layers.Dense(output_dimensions, activation = 'softmax')
  ])
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model


In [ ]:
def evaluate_model(X_train, X_test, y_train, y_test, model, epochs = 100, bs = 8):
  #batch size = # of examples that we are going to feed forward and then backpropogate the error and update their weights accoringly at
  # a time
  model.fit(X_train,y_train,epochs = epochs, batch_size = bs, verbose = 0)
  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis = 1)
  y_test_classes = np.argmax(y_test, axis = 1)
  loss, acurracy = model.evaluate(X_test, y_test, verbose = 0)
  precision = precision_score(y_test_classes, y_pred_classes, average = "macro")
  recall = recall_score(y_test_classes, y_pred_classes, average = "macro")
  f1 = f1_score(y_test_classes, y_pred_classes, average = "macro")
  cm = confusion_matrix(y_test_classes, y_pred_classes)
  return loss, acurracy, precision, recall, f1, cm


In [ ]:
irisds = load_iris()
X, y = irisds.data, irisds.target

k = 5
hidden_layer_nodes = [4, 8, 12, 20]
epochs = 200
batch_size = 16

for hidden_nodes in hidden_layer_nodes:
  print(f"number of hidden layers nodes {hidden_nodes}")
  all_metrics = cross_val_with_hyperparams(X, y, k, hidden_nodes, epochs, batch_size)
  mean_loss = np.mean([ m['loss'] for m in all_metrics])
  mean_accuracy = np.mean([ m['accuracy'] for m in all_metrics])
  mean_precision = np.mean([ m['precision'] for m in all_metrics])
  mean_recall = np.mean([ m['recall'] for m in all_metrics])
  mean_f1 = np.mean([ m['f1'] for m in all_metrics])
  print(f"mean_loss is {mean_loss}")
  print(f"mean_accuracy is {mean_accuracy}")
  print(f"mean_precision is {mean_precision}")
  print(f"mean_recall is {mean_recall}")
  print(f"mean_f1 is {mean_f1}")


number of hidden layers nodes 4
1/1 [==============================] - 0s 42ms/step
mean_loss is 0.23100544810295104
mean_accuracy is 0.9533333301544189
mean_precision is 0.9539393939393939
mean_recall is 0.9533333333333334
mean_f1 is 0.9533166248955723
number of hidden layers nodes 8
1/1 [==============================] - 0s 42ms/step
mean_loss is 0.13722097128629684
mean_accuracy is 0.9599999785423279
mean_precision is 0.9646464646464648
mean_recall is 0.96
mean_f1 is 0.9597984861142755
number of hidden layers nodes 12
1/1 [==============================] - 0s 42ms/step
mean_loss is 0.11052779853343964
mean_accuracy is 0.9466666460037232
mean_precision is 0.9531002331002331
mean_recall is 0.9466666666666667
mean_f1 is 0.9470047098422384
number of hidden layers nodes 20
1/1 [==============================] - 0s 43ms/step
mean_loss is 0.08963831663131713
mean_accuracy is 0.9599999785423279
mean_precision is 0.9646464646464648
mean_recall is 0.96
mean_f1 is 0.9597984861142755


In [ ]:
def cross_val_with_hyperparams(X, y, k, hidden_layer_nodes, epochs = 100, batch_size = 8):
  encoder = OneHotEncoder(sparse_output= False)
  y_onehot = encoder.fit_transform(y.reshape(-1, 1))
  scaler = StandardScaler()
  X = scaler.fit_transform(X)
  input_dimensions = X.shape[1]
  output_dimensions = len(np.unique(y))
  kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 17)
  all_metrics = []
  for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_onehot[train_index], y_onehot[test_index]
    model = create_model(hidden_layer_nodes, input_dimensions, output_dimensions)
    loss, acurracy, precision, recall, f1, cm = evaluate_model(X_train, X_test, y_train, y_test, model, epochs = epochs, bs = batch_size)
    all_metrics.append({"loss": loss, "accuracy": acurracy, "precision": precision, "recall": recall, "f1": f1, "cm": cm })
  return all_metrics



In [ ]:
def create_model(hidden_layer_nodes, input_dimensions, output_dimensions, num_hidden_layers):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(hidden_layer_nodes, activation = "relu", input_dim = input_dimensions))
  for _ in range(num_hidden_layers-1):
    model.add(tf.keras.layers.Dense(hidden_layer_nodes, activation = "relu"))
  model.add(tf.keras.layers.Dense(output_dimensions, activation = "softmax"))
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model


In [ ]:
def cross_val_with_hyperparams(X, y, k, hidden_layer_nodes, hidden_layers, epochs = 100, batch_size = 8):
  encoder = OneHotEncoder(sparse_output= False)
  y_onehot = encoder.fit_transform(y.reshape(-1, 1))
  scaler = StandardScaler()
  X = scaler.fit_transform(X)
  input_dimensions = X.shape[1]
  output_dimensions = len(np.unique(y))
  kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 17)
  all_metrics = []
  for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_onehot[train_index], y_onehot[test_index]
    model = create_model(hidden_layer_nodes, input_dimensions, output_dimensions, hidden_layers)
    loss, acurracy, precision, recall, f1, cm = evaluate_model(X_train, X_test, y_train, y_test, model, epochs = epochs, bs = batch_size)
    all_metrics.append({"loss": loss, "accuracy": acurracy, "precision": precision, "recall": recall, "f1": f1, "cm": cm })
  return all_metrics

In [ ]:
irisds = load_iris()
X, y = irisds.data, irisds.target

k = 5
hidden_layer_nodes = [4, 8, 12, 20]
hidden_layers = [1, 2, 3]
epochs = 200
batch_size = 16
for hidden_layer in hidden_layers:
  print(f"number of hidden layers {hidden_layer}")
  for hidden_nodes in hidden_layer_nodes:
    print(f"number of hidden layers nodes {hidden_nodes}")
    all_metrics = cross_val_with_hyperparams(X, y, k, hidden_nodes, hidden_layer, epochs, batch_size)
    mean_loss = np.mean([ m['loss'] for m in all_metrics])
    mean_accuracy = np.mean([ m['accuracy'] for m in all_metrics])
    mean_precision = np.mean([ m['precision'] for m in all_metrics])
    mean_recall = np.mean([ m['recall'] for m in all_metrics])
    mean_f1 = np.mean([ m['f1'] for m in all_metrics])
    print(f"mean_loss is {mean_loss}")
    print(f"mean_accuracy is {mean_accuracy}")
    print(f"mean_precision is {mean_precision}")
    print(f"mean_recall is {mean_recall}")
    print(f"mean_f1 is {mean_f1}")

number of hidden layers 1
number of hidden layers nodes 4
1/1 [==============================] - 0s 77ms/step


1/1 [==============================] - 0s 53ms/step


mean_loss is 0.192956805229187
mean_accuracy is 0.9333333134651184
mean_precision is 0.9396322196322195
mean_recall is 0.9333333333333332
mean_f1 is 0.932789433088262
number of hidden layers nodes 8


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 46ms/step
mean_loss is 0.12285612970590591
mean_accuracy is 0.9466666460037232
mean_precision is 0.948956228956229
mean_recall is 0.9466666666666667
mean_f1 is 0.9465664160401002
number of hidden layers nodes 12
1/1 [==============================] - 0s 44ms/step
mean_loss is 0.09369186013936996
mean_accuracy is 0.9666666507720947
mean_precision is 0.9684848484848484
mean_recall is 0.9666666666666668
mean_f1 is 0.9666165413533834
number of hidden layers nodes 20
1/1 [==============================] - 0s 47ms/step
mean_loss is 0.0931976318359375
mean_accuracy is 0.9599999785423279
mean_precision is 0.9664335664335664
mean_recall is 0.96
mean_f1 is 0.9603380431755717
number of hidden layers 2
number of hidden layers nodes 4
1/1 [==============================] - 0s 51ms/step
mean_loss is 0.15519867539405824
mean_accuracy is 0.9399999856948853
mean_precision is 0.9502719502719504
mean_recall is 0.9400000000000001
mean_f1 is 0.9392368647988516
numb

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


mean_loss is 0.35402409434318544
mean_accuracy is 0.8333333194255829
mean_precision is 0.7907070707070707
mean_recall is 0.8333333333333333
mean_f1 is 0.7999498746867167
number of hidden layers nodes 8
1/1 [==============================] - 0s 57ms/step
mean_loss is 0.07784879729151725
mean_accuracy is 0.9733333230018616
mean_precision is 0.9745454545454546
mean_recall is 0.9733333333333334
mean_f1 is 0.9732999164578111
number of hidden layers nodes 12
1/1 [==============================] - 0s 58ms/step
mean_loss is 0.14514271430671216
mean_accuracy is 0.9666666507720947
mean_precision is 0.9724941724941726
mean_recall is 0.9666666666666668
mean_f1 is 0.9670214182799995
number of hidden layers nodes 20
1/1 [==============================] - 0s 58ms/step
mean_loss is 0.21472331508994102
mean_accuracy is 0.9466666579246521
mean_precision is 0.9523232323232322
mean_recall is 0.9466666666666667
mean_f1 is 0.9463638895217843


In [ ]:
    all_metrics = cross_val_with_hyperparams(X, y, k, 20, 3, 350, batch_size)
    mean_loss = np.mean([ m['loss'] for m in all_metrics])
    mean_accuracy = np.mean([ m['accuracy'] for m in all_metrics])
    mean_precision = np.mean([ m['precision'] for m in all_metrics])
    mean_recall = np.mean([ m['recall'] for m in all_metrics])
    mean_f1 = np.mean([ m['f1'] for m in all_metrics])
    print(f"mean_loss is {mean_loss}")
    print(f"mean_accuracy is {mean_accuracy}")
    print(f"mean_precision is {mean_precision}")
    print(f"mean_recall is {mean_recall}")
    print(f"mean_f1 is {mean_f1}")

1/1 [==============================] - 0s 58ms/step
mean_loss is 0.192274259775877
mean_accuracy is 0.9466666460037232
mean_precision is 0.9496969696969696
mean_recall is 0.9466666666666667
mean_f1 is 0.9469172932330828


In [ ]:
def create_model(hidden_layer_nodes, input_dimensions, output_dimensions, num_hidden_layers, drop_out = 0.5):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(hidden_layer_nodes, activation = "relu", input_dim = input_dimensions))
  model.add(tf.keras.layers.Dropout(drop_out))
  for _ in range(num_hidden_layers-1):
    model.add(tf.keras.layers.Dense(hidden_layer_nodes, activation = "relu"))
    model.add(tf.keras.layers.Dropout(drop_out))
  model.add(tf.keras.layers.Dense(output_dimensions, activation = "softmax"))
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
irisds = load_iris()
X, y = irisds.data, irisds.target

k = 5
hidden_layer_nodes = [4, 8, 12, 20]
hidden_layers = [1, 2, 3]
epochs = 200
batch_size = 16
for hidden_layer in hidden_layers:
  print(f"number of hidden layers {hidden_layer}")
  for hidden_nodes in hidden_layer_nodes:
    print(f"number of hidden layers nodes {hidden_nodes}")
    all_metrics = cross_val_with_hyperparams(X, y, k, hidden_nodes, hidden_layer, epochs = epochs, batch_size = batch_size)
    mean_loss = np.mean([ m['loss'] for m in all_metrics])
    mean_accuracy = np.mean([ m['accuracy'] for m in all_metrics])
    mean_precision = np.mean([ m['precision'] for m in all_metrics])
    mean_recall = np.mean([ m['recall'] for m in all_metrics])
    mean_f1 = np.mean([ m['f1'] for m in all_metrics])
    print(f"mean_loss is {mean_loss}")
    print(f"mean_accuracy is {mean_accuracy}")
    print(f"mean_precision is {mean_precision}")
    print(f"mean_recall is {mean_recall}")
    print(f"mean_f1 is {mean_f1}")

number of hidden layers 1
number of hidden layers nodes 4
1/1 [==============================] - 0s 49ms/step
mean_loss is 0.4142501771450043
mean_accuracy is 0.8799999833106995
mean_precision is 0.8968039368039367
mean_recall is 0.8800000000000001
mean_f1 is 0.8772505859704676
number of hidden layers nodes 8
1/1 [==============================] - 0s 48ms/step
mean_loss is 0.24183949530124665
mean_accuracy is 0.9466666460037232
mean_precision is 0.9531002331002331
mean_recall is 0.9466666666666667
mean_f1 is 0.9461561832971175
number of hidden layers nodes 12
1/1 [==============================] - 0s 54ms/step
mean_loss is 0.2066779464483261
mean_accuracy is 0.9333333134651184
mean_precision is 0.9462626262626262
mean_recall is 0.9333333333333333
mean_f1 is 0.9310609857978278
number of hidden layers nodes 20
1/1 [==============================] - 0s 53ms/step
mean_loss is 0.12669719457626344
mean_accuracy is 0.9666666507720947
mean_precision is 0.9684848484848484
mean_recall is 0.96666

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 57ms/step
mean_loss is 0.5634447455406189
mean_accuracy is 0.8866666555404663
mean_precision is 0.8572390572390572
mean_recall is 0.8866666666666665
mean_f1 is 0.8642262221209588
number of hidden layers nodes 8
1/1 [==============================] - 0s 79ms/step
mean_loss is 0.3224530667066574
mean_accuracy is 0.9266666531562805
mean_precision is 0.9359692159692159
mean_recall is 0.9266666666666665
mean_f1 is 0.9254720133667501
number of hidden layers nodes 12
1/1 [==============================] - 0s 51ms/step
mean_loss is 0.1939221054315567
mean_accuracy is 0.95333331823349
mean_precision is 0.9595959595959597
mean_recall is 0.9533333333333334
mean_f1 is 0.9530138477506899
number of hidden layers nodes 20
1/1 [==============================] - 0s 58ms/step
mean_loss is 0.11651714444160462
mean_accuracy is 0.95333331823349
mean_precision is 0.9573737373737373
mean_recall is 0.9533333333333334
mean_f1 is 0.95314852788537
number of hidden layers

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 61ms/step
mean_loss is 0.6801216959953308
mean_accuracy is 0.8066666722297668
mean_precision is 0.7998364598364598
mean_recall is 0.8066666666666666
mean_f1 is 0.772292108762697
number of hidden layers nodes 8
1/1 [==============================] - 0s 71ms/step
mean_loss is 0.3425520181655884
mean_accuracy is 0.9133333206176758
mean_precision is 0.9233359233359234
mean_recall is 0.9133333333333333
mean_f1 is 0.9125033643811407
number of hidden layers nodes 12
1/1 [==============================] - 0s 62ms/step
mean_loss is 0.2773332953453064
mean_accuracy is 0.9266666650772095
mean_precision is 0.9356060606060608
mean_recall is 0.9266666666666665
mean_f1 is 0.9259765575555049
number of hidden layers nodes 20
1/1 [==============================] - 0s 67ms/step
mean_loss is 0.13466150015592576
mean_accuracy is 0.9599999785423279
mean_precision is 0.9624242424242425
mean_recall is 0.96
mean_f1 is 0.9599331662489557
